In [8]:
import pandas as pd
import numpy as np
import os
from detect_peaks import detect_peaks , _plot

In [9]:
def analysis(data , f):
    depth_data = data.ix[:,:]

    tension_data = data.ix[:,5:7]
    depth_data['tension'] = tension_data['Tension(Y)']
    depth_data['Depth(Y)'] = (np.ceil(depth_data["Depth(Y)"]/20)*20).astype(int)

    depth_data['block'] = (depth_data["Depth(Y)"].shift(1) != depth_data["Depth(Y)"]).astype(int).cumsum()

    l = depth_data.reset_index().groupby(["Depth(Y)","block"]) # Finding ranges where depth is constant

    filtered_depth_indices = filter(lambda x : len(l.indices[x]) > 100 , l.indices)

    sorted_depth_indices = sorted(filtered_depth_indices , key = lambda x : len(l.indices[x]) , reverse = True)

    m,n = sorted_depth_indices[00]
    t = np.where((depth_data["Depth(Y)"] == m) & (depth_data['block'] == n) )
    m = tension_data[t[0][0]:t[0][-1]].max()
    set_df = pd.DataFrame(columns = ["start" , "end" , "max_tension"])
    nd = []

    def tensions():
        l = []
        for e in sorted_depth_indices:
            m,n = e
            t = np.where((depth_data["Depth(Y)"] == m) & (depth_data['block'] == n) )[0]
            max = tension_data[t[0]:t[-1]]['Tension(Y)'].max()
            max_index = np.where((depth_data['tension'] == max) & (depth_data['block'] == n))
            nd.append(max)
            set_df
            l.extend(max_index[0])
        return l

    t = tensions()
    set_df

    data = []
    for e in range(len(depth_data)):
        val = 0
        if e in t:
            val = depth_data['tension'][e]
        data.append(val)

    max_tension = depth_data['tension'].max()
    max_tension_index = np.where(depth_data['tension'] == max_tension)[0][0]
    max_tension = (max_tension_index , max_tension)

    max_speed = depth_data['Depth_dt(Y)'].max()
    max_speed_index = np.where(depth_data['Depth_dt(Y)'] == max_speed)[0][0]
    max_speed = (max_speed_index , max_speed)

    id = detect_peaks(data , mph = tension_data['Tension(Y)'].median())
    _plot(tension_data['Tension(Y)'] , depth_data['Depth(Y)'] , f , max_speed , max_tension, 1500 , mpd = 1 , threshold = 0,edge = "rising" , valley = False , ax = None ,ind =  id )

In [10]:
for e in os.listdir("data"):
    data = pd.read_csv("data/"+e)
    analysis(data , e+".png")

/home/shikhar/Documents/vij/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
